In [ ]:
# load our graph data from drive, if you run it locally, ignore this
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install networkx

In [ ]:
import pickle
import numpy as np
import scipy as sp
import networkx
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch import nn

In [ ]:
file = open('/content/drive/MyDrive/df_agg (1).pkl','rb')
df_agg = pickle.load(file)
df_agg.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17544 entries, 2012-01-01 00:00:00 to 2013-12-31 23:00:00
Columns: 5558 entries, 2 to 5564
dtypes: float64(5558)
memory usage: 744.1 MB


In [ ]:
file = open('/content/drive/MyDrive/windows_agg_ids (2).pkl','rb')
windows = pickle.load(file)
windows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5558 entries, 0 to 5557
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype          
---  ------       --------------  -----          
 0   LCLid        5558 non-null   int64          
 1   Enabled At   5558 non-null   datetime64[ns] 
 2   Disabled At  5558 non-null   datetime64[ns] 
 3   Duration     5558 non-null   timedelta64[ns]
dtypes: datetime64[ns](2), int64(1), timedelta64[ns](1)
memory usage: 217.1 KB


# Filter nodes that are active from 2012-11-01(7320) to 2013-11-01(16079)

In [ ]:
from tqdm import tqdm

In [ ]:
timestampidx = 7320
active_nodes = df_agg.columns[df_agg.loc[df_agg.index[timestampidx], :].notna()].to_numpy()
for timestampidx in tqdm(range(7321, 16080)):
  act = df_agg.columns[df_agg.loc[df_agg.index[timestampidx], :].notna()].to_numpy()
  active_nodes = np.intersect1d(act, active_nodes)


100%|██████████| 8759/8759 [00:50<00:00, 171.92it/s]


In [ ]:
nodes_selected = active_nodes
np.count_nonzero(np.isnan(df_agg[3][7320:16081].to_numpy()))

0

In [ ]:
data = []
for node in nodes_selected:
  data.append(df_agg[node][7320:16081].to_numpy())

In [ ]:
!pip install dtw

  Preparing metadata (setup.py) ... done
  Created wheel for dtw: filename=dtw-1.4.0-py3-none-any.whl size=5299 sha256=f60981ae37f7fc82d4f182fb9583cdff76dbb62b55d02af1d76081cf28a98e3c
  Stored in directory: /root/.cache/pip/wheels/71/31/62/5e67f6e31521b8cc1652aa58b1ed0c892514d01eeee85fd49c
Successfully built dtw


In [ ]:
from dtw import accelerated_dtw
from scipy.spatial.distance import euclidean

In [ ]:
def min_max_normalize(ts):
    return (ts - np.min(ts)) / (np.max(ts) - np.min(ts))
def euclidean_similarity(ts1, ts2):
    return euclidean(ts1, ts2)


def compute_similarities(data):
    n = data.shape[0]
    similarity_matrix = np.zeros((n, n))

    for i in tqdm(range(n)):
        for j in range(i, n):
            ts1, ts2 = data[i], data[j]


            ts1, ts2 = min_max_normalize(ts1), min_max_normalize(ts2)

            similarity = euclidean_similarity(ts1, ts2)

            similarity_matrix[i, j] = similarity
            similarity_matrix[j, i] = similarity

    return similarity_matrix
similarity_matrix = compute_similarities(np.array(data))

Streaming output truncated to the last 5000 lines.
100%|██████████| 531/531 [00:00<00:00, 6040.91it/s]

100%|██████████| 523/523 [00:00<00:00, 5959.87it/s]

100%|██████████| 495/495 [00:00<00:00, 5852.22it/s]

100%|██████████| 493/493 [00:00<00:00, 5646.37it/s]

100%|██████████| 483/483 [00:00<00:00, 5548.84it/s]

100%|██████████| 472/472 [00:00<00:00, 5372.51it/s]

100%|██████████| 469/469 [00:00<00:00, 5233.65it/s]

100%|██████████| 466/466 [00:00<00:00, 5976.14it/s]

100%|██████████| 464/464 [00:00<00:00, 5662.62it/s]

100%|██████████| 448/448 [00:00<00:00, 5627.56it/s]

100%|██████████| 444/444 [00:00<00:00, 5628.37it/s]

100%|██████████| 420/420 [00:00<00:00, 5362.11it/s]

100%|██████████| 416/416 [00:00<00:00, 5319.54it/s]

100%|██████████| 414/414 [00:00<00:00, 5441.62it/s]

100%|██████████| 412/412 [00:00<00:00, 5063.28it/s]

100%|██████████| 410/410 [00:00<00:00, 5702.49it/s]

100%|██████████| 408/408 [00:00<00:00, 4438.45it/s]

100%|██████████| 401/401 [00:00<00:00, 4837.14it

In [31]:
similarity_matrix

array([[ 0.        , 20.38499449, 21.73524107, ..., 21.70005769,
        23.21736555, 23.5647041 ],
       [20.38499449,  0.        ,  8.40958389, ...,  9.88383767,
        14.90072681, 14.07917511],
       [21.73524107,  8.40958389,  0.        , ...,  9.0002583 ,
        13.93403278, 14.36534656],
       ...,
       [21.70005769,  9.88383767,  9.0002583 , ...,  0.        ,
        12.61809598, 14.45792892],
       [23.21736555, 14.90072681, 13.93403278, ..., 12.61809598,
         0.        , 15.65332821],
       [23.5647041 , 14.07917511, 14.36534656, ..., 14.45792892,
        15.65332821,  0.        ]])

In [33]:
import pickle
file = open('similarity_matrix', 'wb')
pickle.dump(similarity_matrix, file)

Hence we only have 3708 nodes that are active from all timestamp in this period. If we would like more nodes, we could make interpolations.

# Data preparation

# MGTNN

paper:https://arxiv.org/abs/2005.11650

code: https://github.com/nnzhan/MTGNN/tree/master

A interesting framework which uses time-series data to make predictions.
The most impotant part is that this framework learn both the graph structure and GNN. That is, in their data they don't have a predefined graph structure at  first (like us), so they have a graph learning layer which learns the adjacency matrix from the time-series data. The learned adjacency matrix then is input to the graph convolution layer and temporal layer.

# Graph learning layer

In their work they choose to learn directed graph.
Code below shows the example an un-directed graph learning layer.

In [ ]:
class graph_undirected(nn.Module):
    def __init__(self, nnodes, k, dim, device, alpha=3, static_feat=None):
        super(graph_undirected, self).__init__()
        self.nnodes = nnodes
        if static_feat is not None:
            xd = static_feat.shape[1]
            self.lin1 = nn.Linear(xd, dim)
        else:
            self.emb1 = nn.Embedding(nnodes, dim)
            self.lin1 = nn.Linear(dim,dim)

        self.device = device
        self.k = k
        self.dim = dim
        self.alpha = alpha
        self.static_feat = static_feat

    def forward(self, idx):
        if self.static_feat is None:
            nodevec1 = self.emb1(idx)
            nodevec2 = self.emb1(idx)
        else:
            nodevec1 = self.static_feat[idx,:]
            nodevec2 = nodevec1

        nodevec1 = torch.tanh(self.alpha*self.lin1(nodevec1))
        nodevec2 = torch.tanh(self.alpha*self.lin1(nodevec2))

        a = torch.mm(nodevec1, nodevec2.transpose(1,0))
        adj = F.relu(torch.tanh(self.alpha*a))
        mask = torch.zeros(idx.size(0), idx.size(0)).to(self.device)
        mask.fill_(float('0'))
        s1,t1 = adj.topk(self.k,1)
        mask.scatter_(1,t1,s1.fill_(1))
        adj = adj*mask
        return adj


**Thought**: if we do not learn adj matrix in the framework, is it ok if we build the adj matrix from the similarity between time series data (maybe computation expensive)

# GCN and TCN
For GCN layer they use mix hop propogation.
For TCN layer they use dilated inception layer.
Those two are fancy but may not easy to use. We could choose simplier one.